In [1]:
!pip install sentencepiece

In [4]:
from torch.utils import tensorboard #pip install tensorboard

In [5]:
from transformers import pipeline, MarianTokenizer, MarianMTModel,DistilBertTokenizer, DistilBertForSequenceClassification

In [6]:
#This code translates user input to french and gets the sentiment at the same time, it incorporates translation and sentiment analysis.

#using pre-trained translation model
translation_model_name = "Helsinki-NLP/opus-mt-en-fr"
translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
translation_model = MarianMTModel.from_pretrained(translation_model_name)

#Using sentiment model
#sentiment_analysis_pipeline = pipeline("sentiment-analysis")
sentiment_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
sentiment_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
sentiment_analysis_pipeline = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=sentiment_tokenizer)


/home/lkk/miniconda3/envs/mypy310/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Using /home/lkk/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Creating extension directory /home/lkk/.cache/torch_extensions/py310_cu118/cuda_kernel...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/lkk/.cache/torch_extensions/py310_cu118/cuda_kernel/build.ninja...
Building extension module cuda_kernel...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/4] /home/lkk/miniconda3/envs/mypy310/bin/nvcc  -DTORCH_EXTENSION_NAME=cuda_kernel -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/lkk/miniconda3/envs/mypy310/lib/python3.10/site-packages/torch/include -isystem /home/lkk/miniconda3/envs/mypy310/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /home/lkk/miniconda3/envs/mypy310/lib/python3.10/site-packages/torch/include/TH -isystem /home/lkk/miniconda3/envs/mypy310/lib/python3.10/site-packages/torch/include/THC -isystem /home/lkk/miniconda3/envs/mypy310/include -isystem /home/lkk/miniconda3/envs/mypy310/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencode=arch=compute_89,code=compute_89 -gencode=arch=compute_89,code=sm_89 --compiler-options '-fPIC' -std=c++17 

Failed to load CUDA kernels. Mra requires custom CUDA kernels. Please verify that compatible versions of PyTorch and CUDA Toolkit are installed: Error building extension 'cuda_kernel'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[4/4] c++ cuda_kernel.cuda.o cuda_launch.cuda.o torch_extension.o -shared -L/home/lkk/miniconda3/envs/mypy310/lib/python3.10/site-packages/torch/lib -lc10 -lc10_cuda -ltorch_cpu -ltorch_cuda -ltorch -ltorch_python -L/home/lkk/miniconda3/envs/mypy310/lib64 -lcudart -o cuda_kernel.so
FAILED: cuda_kernel.so 
c++ cuda_kernel.cuda.o cuda_launch.cuda.o torch_extension.o -shared -L/home/lkk/miniconda3/envs/mypy310/lib/python3.10/site-packages/torch/lib -lc10 -lc10_cuda -ltorch_cpu -ltorch_cuda -ltorch -ltorch_python -L/home/lkk/miniconda3/envs/mypy310/lib64 -lcudart -o cuda_kernel.so
/usr/bin/ld: cannot find -lcudart: No such file or directory
collect2: error: ld returned 1 exit status
ninja: build stopped: subcommand failed.


In [7]:
print("Chatbot: Hello! i can help you translate to french, please enter a sentence?")

while True:
    user_input = input("You: ")
#checks exit functionality : chatbot stops when user uses following in a sentence
    if any(token in user_input.lower() for token in ['exit', 'bye']):
        print("Chatbot: Goodbye! Have a great day!")
        break

 # Perform sentiment analysis on English input
    sentiment_output = sentiment_analysis_pipeline(user_input)[0]
    sentiment_label = sentiment_output['label']

  # Translate English input to French
    translated_input = translation_tokenizer.encode(user_input, return_tensors="pt")
    translated_output = translation_model.generate(translated_input, max_length=100, num_return_sequences=1)
    translated_text = translation_tokenizer.decode(translated_output[0], skip_special_tokens=True)

  # Display chatbot response
    response = f"Chatbot: Sentiment: {sentiment_label}. Translated text in French: {translated_text}"
    print(response)

Chatbot: Hello! i can help you translate to french, please enter a sentence?
Chatbot: Sentiment: POSITIVE. Translated text in French: Bonjour, bonjour.
Chatbot: Sentiment: POSITIVE. Translated text in French: Bonjour Français, bonjour.
Chatbot: Sentiment: POSITIVE. Translated text in French: J'aime les hamburgers.
Chatbot: Goodbye! Have a great day!


In [9]:
# Initialising conversation history
conversation_history = []

print("Chatbot: Hello! How can I assist you today?")


Chatbot: Hello! How can I assist you today?


In [10]:
import requests
while True:
    user_input = input("You: ")
  #exit mechanism
    if any(token in user_input.lower() for token in ['exit', 'bye']):
        print("Chatbot: Goodbye! Have a great day!")
        break

    if 'weather' in user_input.lower():
        print("Chatbot: Sure! Please provide the city name:")
        city = input("You: ")
        weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
        response = requests.get(weather_url).json()
        if response.get('main') and response.get('weather'):
            temperature = response['main']['temp']
            description = response['weather'][0]['description']
            response = f"Chatbot: The weather in {city} is currently {description} with a temperature of {temperature:.2f}°C."
        else:
            response = "Chatbot: I'm sorry, I couldn't fetch the weather information for that city."
        print(response)  # Print the weather response
    else:
        sentiment = sentiment_analysis_pipeline(user_input)[0]
        sentiment_label = sentiment['label']
        sentiment_score = sentiment['score']

        if sentiment_label == 'POSITIVE':
            response = "Chatbot: It sounds like you're feeling positive! How can I help you further?"
        elif sentiment_label == 'NEGATIVE':
            response = "Chatbot: I'm sorry to hear that. How can I assist you in improving your mood?"
        else:
            response = "Chatbot: Thanks for sharing. Is there something specific you'd like to discuss?"

        conversation_history.append({"user_input": user_input, "sentiment": sentiment_label, "chatbot_response": response})
        print(response)

Chatbot: It sounds like you're feeling positive! How can I help you further?
Chatbot: Sure! Please provide the city name:
Chatbot: I'm sorry, I couldn't fetch the weather information for that city.
Chatbot: It sounds like you're feeling positive! How can I help you further?
Chatbot: Sure! Please provide the city name:
Chatbot: I'm sorry, I couldn't fetch the weather information for that city.
Chatbot: Sure! Please provide the city name:
Chatbot: The weather in san jose is currently broken clouds with a temperature of 23.34°C.
Chatbot: Goodbye! Have a great day!
